In [1]:
import numpy as np
import os
import cv2
import pickle
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
def mask_overlap(mask11, mask22):
###############################################
###############################################
    """
    This function calculate region IOU when masks are
    inside different boxes
    Returns:
        intersection over unions of this two masks
    """
    mask1 = mask11.copy()
    mask2 = mask22.copy()
    
    mask1[mask1>=1] = 1
    mask2[mask2>=1] = 1
    
    #plt.subplot(1,2,1) 
    #plt.imshow(mask1)
    #plt.subplot(1,2,2) 
    #plt.imshow(mask2)
    
    green = (255,255,255)
    #box1 / box2 are the bounding box of mask1 / mask2 
    blob1 = np.where(mask1>=1)
    box1x1 = min(blob1[1])
    box1x2 = max(blob1[1])
    box1y1 = min(blob1[0])
    box1y2 = max(blob1[0])
    
    #cv2.rectangle(mask1,(box1x1,box1y1),(box1x2,box1y2),0.5,1)
    
    #plt.subplot(1,2,1) 
    #plt.imshow(mask1)
    #plt.subplot(1,2,2) 
    #plt.imshow(mask2)
    
    #print box1x1,box1x2,box1y1,box1y2
    
    blob2 = np.where(mask2>=1)
    box2x1 = min(blob2[1])
    box2x2 = max(blob2[1])
    box2y1 = min(blob2[0])
    box2y2 = max(blob2[0])
    
    #cv2.rectangle(mask2,(box2x1,box2y1),(box2x2,box2y2),0.5,1)
    
    x1 = max(box1x1 , box2x1)
    y1 = max(box1y1 , box2y1)
    x2 = min(box1x2 , box2x2)
    y2 = min(box1y2 , box2y2)

    #######################
    
    if x1 > x2 or y1 > y2:
        return 0
    w = x2 - x1 + 1
    h = y2 - y1 + 1
    # get masks in the intersection part
    inter_maska = mask1[y1: y1 + h, x1:x1 + w]
    inter_maskb = mask2[y1: y1 + h, x1:x1 + w]

    #print inter_maska
    #print inter_maskb
    
    #plt.subplot(2,2,1) 
    #plt.imshow(inter_maska)
    #plt.subplot(2,2,2) 
    #plt.imshow(inter_maskb)    
    
    assert inter_maska.shape == inter_maskb.shape
    inter = np.logical_and(inter_maskb, inter_maska).sum()
    #print inter_maskb
    #print inter_maska
    union = mask1.sum() + mask2.sum() - inter
    
    if union < 1.0:
        return 0
    return float(inter) / float(union)


In [3]:
def voc_ap(rec, prec, use_07_metric=False):
    """
    average precision calculations
    [precision integrated to recall]
    :param rec: recall
    :param prec: precision
    :param use_07_metric: 2007 metric is 11-recall-point based AP
    :return: average precision
    """
    if use_07_metric:
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap += p / 11.
    else:
        # append sentinel values at both ends
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        # compute precision integration ladder
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        # look for recall value changes
        i = np.where(mrec[1:] != mrec[:-1])[0]

        # sum (\delta recall) * prec
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

In [4]:
def ClassifyColor(Red,Green,Blue):
    if Blue == 255:
        if Green == 0 and Red % 10 == 0 and Red != 0:
            return 'CubeBody'
        elif Red % 10 == 0 and Green == Red and Red != 0:
            return 'CubeFace'
    elif Blue == 200:
        if Green == 0 and Red % 10 == 0 and Red != 0:
            return 'CylinderBody'
        elif Red % 10 == 0 and Green == Red and Red != 0:
            return 'CylinderFace'
    elif Blue == 150:
        if Green == 0 and Red % 10 == 0 and Red != 0:
            #return 'Handle'
            return 'Grip'
    else:
        if Red == 0 and Green == 0 and Blue == 0:
            return 'Background'
    return 'Invalid'

In [5]:
def ParseMask(img,h,w):
    part = []
    partlabel = []
    visited_color = []
    
    blue_channel = img[:,:,0]
    pixel = blue_channel.nonzero()
    for i in range(0,len(pixel[0])):
        color = img[pixel[0][i],pixel[1][i]]
        color_string =  str("%03d" % color[0]) + str("%03d" % color[1]) +str("%03d" % color[2])
        if color_string not in visited_color: 
            visited_color.append(color_string)
            label = ClassifyColor(color[2],color[1],color[0])
            if label!= 'Background' and label!='Invalid':
                lower = np.array(color, dtype = "uint8")#颜色下限  
                upper = np.array(color, dtype = "uint8")#颜色上限  

                #plt.imshow(fcis_img)
                mask = cv2.inRange(img, lower, upper) 
                output = cv2.bitwise_and(img,img,mask=mask)
                output = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
                output[output>=1] = 1

                part.append(output)
                partlabel.append(label)
    
    return part, partlabel

In [6]:
import os
from glob import glob 
def search_file(pattern, search_path=os.environ['PATH'], pathsep=os.pathsep):
    for path in search_path.split(os.pathsep):
        for match in glob(os.path.join(path, pattern)):
            yield match

In [7]:
def mask_overlap_box(box1, box2, mask1, mask2):
    """
    This function calculate region IOU when masks are
    inside different boxes
    Returns:
        intersection over unions of this two masks
    """
    #print(mask1.shape)
    #print(mask2.shape)
    
    
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    if x1 > x2 or y1 > y2:
        return 0
    w = x2 - x1 + 1
    h = y2 - y1 + 1
    # get masks in the intersection part
    start_ya = y1 - box1[1]
    start_xa = x1 - box1[0]
    inter_maska = mask1[start_ya: start_ya + h, start_xa:start_xa + w]

    start_yb = y1 - box2[1]
    start_xb = x1 - box2[0]
    inter_maskb = mask2[start_yb: start_yb + h, start_xb:start_xb + w]
    
    #plt.subplot(2,2,1) 
    #plt.imshow(inter_maska)
    #plt.subplot(2,2,2) 
    #plt.imshow(inter_maskb) 
    
    #print(inter_maska.shape)
    #print(inter_maskb.shape)
    
    assert inter_maska.shape == inter_maskb.shape

    inter = np.logical_and(inter_maskb, inter_maska).sum()
    union = mask1.sum() + mask2.sum() - inter
    if union < 1.0:
        return 0
    return float(inter) / float(union)

In [8]:
def bbox(part):
    mask = part.copy()    
    mask[mask>=1] = 1
    blob = np.where(mask>=1)
    x1 = min(blob[1])
    x2 = max(blob[1])
    y1 = min(blob[0])
    y2 = max(blob[0])
    
    box = [x1, y1, x2, y2]
    inst =  mask[y1:y2+1,x1:x2+1]
    
    return inst,box

In [9]:
def FetchScore(matches, part):
    maxoverlap = -1000
    targetscore = 0
    bestmatch = []
    my_inst,my_instbox  = bbox(part)
    for oneinst in matches:
        if 'box' not in oneinst:
            score = oneinst[oneinst.rindex('_')+1:oneinst.index('.npy')]
            score = float(score)
            inst = np.load(oneinst)
            instbox = np.load(oneinst[:oneinst.index('.npy')]+'_box.npy')

            ov = mask_overlap_box(instbox,my_instbox,inst,my_inst)
            if ov>maxoverlap:
                targetscore = score
                maxoverlap = ov
                bestmatch = inst
                
    #plt.figure()
    #plt.imshow(my_inst)
    #plt.figure()
    #plt.imshow(bestmatch) 
    #            
    return targetscore

In [10]:
def GetAP(part,label,gt_part,gt_part_label,oneclass,imgname):

    test_label = classes.get(oneclass)    
    num_pred = 0
    for cls in label:
        if cls == classes.get(oneclass):
            num_pred = num_pred+1
            
    fp = np.zeros(num_pred)
    tp = np.zeros(num_pred)
    ov_areas = np.zeros(num_pred)    
    scores = np.zeros(num_pred)
    
    gt_cls_num = 0
    for cls in gt_part_label:
        if cls == classes.get(oneclass):
            gt_cls_num = gt_cls_num+1
            
    #print "instance count: "+ str(num_pred)
    gt_detect_list = np.zeros((len(gt_part), 1))
    
    all_ov = []
    
    idx = 0
    
    for i in range(0,len(part)):
        if label[i] != test_label:
            continue
        
        ###This instance is going to be in tp or fp, need score!!
        ###find top score candidates
        realname = imgname[:imgname.index('.')]
        queryfolder = fcis_path[:len(fcis_path)-1] +'_SCORE/' + classes.get(oneclass) +'/'
        queryname = realname +'_'+ '*'+ '.npy'  
        matches = list(search_file(queryname,queryfolder))
        scores[idx] = FetchScore(matches,part[i])
        if label[i] not in gt_part_label: #this label  is not in ground truth
            fp[idx] = 1
            continue
        cur_overlap = -1000
        cur_overlap_ind = -1
        for j in range(0,len(gt_part)):
            if gt_part_label[j] == label[i]:
                ov = mask_overlap(gt_part[j], part[i])
                if ov > cur_overlap:
                    cur_overlap = ov
                    cur_overlap_ind = j                    
        if cur_overlap == -1000:
            print('overlap==-1000')
            cur_overlap = 0
        ov_areas[idx]=cur_overlap
        if cur_overlap >= ov_thresh:
            if gt_detect_list[cur_overlap_ind]:
                fp[idx] = 1
            else:
                tp[idx] = 1
                gt_detect_list[cur_overlap_ind] = 1
        else:
                fp[idx] = 1
        idx = idx+1
    return tp,fp,ov_areas,scores,gt_cls_num

In [11]:
def GetAPwithOV(part,label,part_ov,part_ov_id,part_score, gt_part, gt_part_label,oneclass,imgname):
    test_label = classes.get(oneclass)    
    num_pred = 0
    for cls in label:
        if cls == classes.get(oneclass):
            num_pred = num_pred+1
            
    fp = np.zeros(num_pred)
    tp = np.zeros(num_pred)
    
    gt_cls_num = 0
    for cls in gt_part_label:
        if cls == classes.get(oneclass):
            gt_cls_num = gt_cls_num+1
            
    #print "instance count: "+ str(num_pred)
    gt_detect_list = np.zeros((len(gt_part), 1))
    
    idx = 0
    
    for i in range(0,len(part)):
        if label[i] != test_label:
            continue       
        score[idx] = part_score[i]
        
        if label[i] not in gt_part_label: #this label  is not in ground truth
            fp[idx] = 1
            continue
               
        cur_overlap = part_ov[i]
        cur_overlap_ind = part_ov_id[i]
        ov_areas[idx]=cur_overlap
        
        if cur_overlap >= ov_thresh:
            if gt_detect_list[cur_overlap_ind]:
                fp[idx] = 1
            else:
                tp[idx] = 1
                gt_detect_list[cur_overlap_ind] = 1
        else:
                fp[idx] = 1
        idx = idx+1

    return tp,fp,ov_areas,scores,gt_cls_num

In [12]:
def savefile(filename,  overlap):
    output = open(filename, 'w')
    #for n in ap:
    #   output.write(str(n)+' ')
    #output.write('\n')
    for n in overlap:
        output.write(str(n)+' ')
    output.close()

In [13]:
def parsename(name):
    #name = 'bod111_0_20_0_255.png'
    blue = name[name.rindex('_')+1:name.rindex('.')]
    #print blue
    temp = name[:name.rindex('_')]
    #print temp
    green = temp[temp.rindex('_')+1:]
    #print green
    temp = name[:temp.rindex('_')]
    #print temp
    red = temp[temp.rindex('_')+1:]
    #print red
    realname = name[:temp.rindex('_')]
    #print realname
    return int(red),int(green),int(blue),realname

In [14]:
##########################################
#####the slowest part
#####separate each image into instances

In [15]:
#classes = {0: 'CubeBody',  1:'CubeFace', 2: 'CylinderBody',3:  'CylinderFace' ,4:'Handle'}
classes = {0: 'CubeBody',  1:'CubeFace', 2: 'CylinderBody',3:  'CylinderFace',4:'Grip' }
num_classes = len(classes)
ov_thresh = 0.5
print(num_classes) 

5


In [25]:
#folderpath = 'ForEvaluationTest_CurrentGeoNetVal/ForEvaluationTest_CurrentGeoNetVal/'
folderpath = 'Exper_FCIS_DCN/'
fcis_path = folderpath + '11_06_FCIS_6class__new_inst/'
gt_path = folderpath + 'gt/'

fcis_list = os.listdir(fcis_path)
gt_list = os.listdir(gt_path)
print(len(fcis_list))
print(len(gt_list))

11699
3482


In [33]:
idx= 0

fcis_part_all = []
fcis_part_label_all = []
fcis_part_score_all = []

imageset_file = os.path.join(folderpath, 'val.txt')
with open(imageset_file, 'r') as f:
    lines = f.readlines()
image_names = [x.strip() for x in lines]

for imgname in image_names:
    print idx
    realname = imgname
    #fcis_img = cv2.imread(fcis_path+realname+'.png', 1)
    #gt_img = cv2.imread(gt_path+realname+'.png', 1)
    
    queryfolder =  fcis_path
    queryname = realname +'_'+ '*'+ '.png'  
    matches = list(search_file(queryname,queryfolder))
    
    part = []
    part_label = []
    
    for oneinstance in matches:
        #print(oneinstance)
        inst = cv2.imread(oneinstance, 1)
        
        if np.sum(inst)==0:
            print(oneinstance)
            continue
        
        blue_channel = inst[:,:,0]
        blue_channel[blue_channel>=1] = 1
        name = oneinstance[oneinstance.rindex('/')+1:]
        r,g,b,name = parsename(name)
        label = ClassifyColor(r,g,b)
        
        print(label)
        #plt.figure()
        #plt.imshow(blue_channel)
        part.append(blue_channel)
        part_label.append(label)
    
    idx = idx+1
    
    fcis_part_all.append(part)
    fcis_part_label_all.append(part_label)
        
    
    
#     h = gt_img.shape[0]
#     w = gt_img.shape[1]
    
#     try:
#         fcis_img.shape[0]
#     except:
#         print("no fcis output")
#         fcis_img =  np.zeros(gt_img.shape,np.uint8)
        

#     print(str(idx) + " " + imgname) 
    
#     fcis_part,fcis_part_label = ParseMask(fcis_img,h,w)
#     fcis_part_all.append(fcis_part)
#     fcis_part_label_all.append(fcis_part_label)

0
CubeBody
CubeFace
1
CubeBody
CubeFace
2
CubeFace
CylinderBody
CubeBody
CylinderFace
3
CubeBody
CubeBody
CubeFace
CubeFace
4
CubeFace
CubeBody
CylinderFace
5
CubeBody
CubeFace
6
CubeFace
CubeBody
CylinderFace
7
CubeFace
CubeBody
8
CubeBody
CubeFace
9
CylinderFace
CubeBody
10
CubeBody
CubeFace
11
CubeFace
CubeBody
12
CubeBody
13
CubeFace
CubeBody
14
CubeFace
CubeBody
15
CubeBody
CubeFace
16
CubeBody
CubeFace
17
CubeBody
CubeFace
18
CubeBody
CubeFace
19
CubeFace
CylinderFace
CubeBody
20
CubeBody
CubeFace
21
CubeBody
CubeFace
22
CubeFace
CubeBody
23
CubeFace
CubeFace
CubeBody
CubeBody
24
CylinderBody
CubeBody
CubeFace
25
CubeFace
CubeBody
CylinderFace
26
CubeBody
CubeFace
CubeBody
CubeFace
27
CubeBody
CubeFace
28
CubeBody
CubeFace
CubeBody
29
CubeFace
CubeBody
30
CubeBody
CubeFace
31
CubeBody
CubeFace
32
CubeBody
CubeFace
CubeFace
CubeBody
33
CubeBody
34
CubeFace
CubeBody
CubeBody
35
CubeFace
CubeBody
36
CubeBody
CubeBody
CubeFace
37
CubeBody
CubeFace
38
CubeBody
CubeFace
39
CubeFace
Cub

KeyboardInterrupt: 

In [27]:
print(len(fcis_part_all))
print(len(fcis_part_label_all))
print(len(image_names))

3474
3474
3474


In [28]:
def GetMask(mask_pkl,box_pkl,shape):
    mask_imgs = []
    for i in range(0,len(box_pkl)):
        mask = mask_pkl[i]
        #print(mask.shape)
        box = box_pkl[i]
        #print(box)
        new_img = np.zeros((shape[0],shape[1]),np.uint8)
        new_img[int(box[1]):int(box[3])+1,int(box[0]):int(box[2])+1] = mask
        
        new_img[new_img>=1] = 1
        #plt.figure()
        #plt.imshow(new_img)
        mask_imgs.append(new_img)
    return mask_imgs

In [29]:
def buildPart(image_names,data_path):
    # 2. Get predict pickle file for this class
    fcis_part_all = []
    fcis_part_label_all = []
    fcis_part_score_all = []
    
    boxes_pkl_all = [[],[],[],[]]
    masks_pkl_all = [[],[],[],[]]
    
    for j in range(0,len(classes)):    
        det_file = os.path.join(data_path, classes.get(j) + '_det.pkl')
        seg_file = os.path.join(data_path, classes.get(j) + '_seg.pkl')
        with open(det_file, 'rb') as f:
            boxes_pkl = pickle.load(f)
        boxes_pkl_all[j] = boxes_pkl
        with open(seg_file, 'rb') as f:
            masks_pkl = pickle.load(f)
        masks_pkl_all[j] = masks_pkl
    
    for i in range(0,len(image_names)):
        print(i)
        inst = []
        label = []
        score = []
        for j in range(0,len(classes)):    
            inst_count = len(boxes_pkl_all[j][i])
            if inst_count == 0:
                continue            
            for x in range(0,inst_count):
                label.append(classes.get(j))
                score.append(boxes_pkl_all[j][i][x][-1])
            
            #print 'instcount: ' +str(inst_count)
            #print 'label count: '+str(len(label))
            #print 'score count:' + str(len(score))
                    
            img = cv2.imread(data_path+image_names[i]+'.png')
            mask = GetMask(masks_pkl_all[j][i],boxes_pkl_all[j][i],img.shape)            
            inst.extend(mask)
            
        fcis_part_all.append(inst)
        fcis_part_score_all.append(score)
        fcis_part_label_all.append(label)
        
 
    return fcis_part_all,fcis_part_score_all,fcis_part_label_all

### save pkl
1. det_file  按img_idx排序，每一行代表当前img的每一个detection，每一个detection为5维，前四维是xmin,ymin,xmax,ymax，最后一维是score
2. seg_file  按img_idx排序，每一行代表当前img的每一个detection对应的segmentation, 在bbox内的mask, 有每一个pixel的score, 改成0/1


In [30]:
def GetPKLline(part,part_label,imgname,cls_name):    
    det_line = []
    seg_line = []
        
    for i in range(0,len(part)):
        oneclass = part_label[i]
        if oneclass == cls_name:
            realname = imgname
            #queryfolder = 'Exper_FCIS_DCN/10_30_FCIS_DCN_ob_noss_SCORE/' + oneclass +'/'
            queryfolder = fcis_path[:len(fcis_path)-6] +'_SCORE/' + oneclass +'/'
            #print queryfolder
            queryname = realname +'_'+ '*'+ '.npy'  
            matches = list(search_file(queryname,queryfolder))
            #print(matches)
            score = FetchScore(matches,part[i])
            #print score
            inst,box = bbox(part[i])
            temp = np.zeros(5)
            temp[0:4] = box
            temp[-1] = score
            
            #inst = cv2.resize(inst.astype(np.float32), (21,21))
            #plt.figure()
            #plt.imshow(inst)
            #inst[inst>0] = 1
            #plt.figure()
            #plt.imshow(inst)
            inst = np.array(inst)
            
            det_line.append(temp)
            seg_line.append(inst)
        
    return det_line,seg_line

In [31]:
fcis_part_score_all = []
fcis_part_ov_all = []
fcis_part_ov_id_all = []
for j in range(0,len(classes)):
    det_file = []
    seg_file = []
    print(classes.get(j))
    for i in range(0,len(fcis_part_all)):
#    for i in range(1000,1010):
        print i
        part = fcis_part_all[i]
        part_label = fcis_part_label_all[i]
        det_line,seg_line= GetPKLline(part,part_label,image_names[i],classes.get(j))
        #print(det_line)
        
        det_file.append(det_line)
        seg_file.append(seg_line)
    
    output_det = open(fcis_path + classes.get(j)+'_det.pkl','wb')
    pickle.dump(det_file,output_det)
    output_det.close()
    
    output_seg = open(fcis_path + classes.get(j)+'_seg.pkl','wb')
    pickle.dump(seg_file,output_seg)
    output_seg.close()

CubeBody
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
2

1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145


175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424


2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220


335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584


2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2274
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2285
2286
2287
2288
2289
2290
2291
2292
2293
2294
2295
2296
2297
2298
2299
2300
2301
2302
2303
2304
2305
2306
2307
2308


368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617


2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2274
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2285
2286
2287
2288
2289
2290
2291
2292
2293
2294
2295
2296
2297
2298
2299
2300
2301
2302
2303
2304
2305
2306
2307
2308
2309
2310
2311
2312
2313
2314
2315
2316
2317
2318
2319
2320
2321
2322
2323
2324
2325
2326
2327
2328
2329
2330
2331
2332
2333
2334


623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872


2337
2338
2339
2340
2341
2342
2343
2344
2345
2346
2347
2348
2349
2350
2351
2352
2353
2354
2355
2356
2357
2358
2359
2360
2361
2362
2363
2364
2365
2366
2367
2368
2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536


In [32]:
print(len(seg_file))
print(len(det_file))

3474
3474


In [66]:
print(det_file)

with open(fcis_path+'CylinderFace_seg.pkl', 'rb') as f:
    masks_pkl = pickle.load(f)

[[], [], [array([  57.        ,  270.        ,   85.        ,  285.        ,
          0.85410053])], [], [array([ 260.        ,   75.        ,  381.        ,   93.        ,
          0.86683512])], [], [array([ 211.        ,  227.        ,  218.        ,  228.        ,
          0.94645309])], [], [], [array([ 178.        ,  213.        ,  271.        ,  254.        ,
          0.91816694])]]


In [54]:
print(len(fcis_part_all))
print(len(fcis_part_score_all))
print(len(fcis_part_ov_all))
print(len(fcis_part_ov_id_all))

3474
11747
11747
11747


In [56]:
#ov_thresh =   0.5 #overlap_threshold - 0.5/0.7....
ov_thresh = np.arange(0.0,1.0,0.05)

In [ ]:
ap_all = []
map_all = []

for one_ov in ov_thresh[10:11]:
    all_tp = [[],[],[],[],[]]
    all_fp = [[],[],[],[],[]]
    all_scores = [[],[],[],[],[]]
    gt_cls_num = np.zeros(num_classes)
    
    for oneclass in range(0,num_classes):
        print(classes.get(oneclass))
        #find all instances in this class
        for i in range(0,len(fcis_part_all)):
            if i %1000 == 0:
                print(i)
                
            part = fcis_part_all[i]
            part_label = fcis_part_label_all[i]
            part_ov = fcis_part_ov_all[i]
            part_ov_id = fcis_part_ov_id_all[i]
            part_score = fcis_part_score_all[i]

            gt_part = gt_part_all[i]
            gt_part_label = gt_part_label_all[i]

            this_cls =  classes.get(oneclass)
            if this_cls in part_label:
                tp, fp , ov_areas , scores ,gt_thiscls_num= GetAPwithOV(part,part_label,part_ov,part_ov_id,part_score,gt_part,gt_part_label,oneclass,gt_list[i])
                #print(scores)
                #print(ov_areas)
                all_tp[oneclass].extend(tp)
                all_fp[oneclass].extend(fp)
                overlap_areas[oneclass].extend(ov_areas)
                all_scores[oneclass].extend(scores)
                gt_cls_num[oneclass] = gt_cls_num[oneclass] + gt_thiscls_num              
        
    ap = np.zeros(num_classes)
    for oneclass in range(0,num_classes):
        ####### 按照score重新对tp 和 fp 排序
        orderidx = np.argsort(all_scores[oneclass])
        orderidx = orderidx[::-1]
        tps_backup = all_tp[oneclass][:]
        fpt_backup = all_fp[oneclass][:]
        or_tps = []
        or_fps = []
        for x in orderidx:
            or_tps.append(all_tp[oneclass][x])
            or_fps.append(all_fp[oneclass][x])
            one_tp = or_tps[:]
            one_fp = or_fps[:]
        ##################################

        #one_tp = all_tp[oneclass]
        #one_fp = all_fp[oneclass]

        num_pos = gt_cls_num[oneclass]
        one_tp = np.cumsum(one_tp)
        one_fp = np.cumsum(one_fp)
        rec = one_tp / np.maximum(float(num_pos),  np.finfo(np.float64).eps)
        #avoid divide by zero in case the first matches a difficult gt
        prec = one_tp / np.maximum(one_fp+one_tp, np.finfo(np.float64).eps)
        ap[oneclass] = voc_ap(rec, prec, True)
        
    print('FCIS')
    print('~~~~~~~~~~~~~~~~~~~~~~~')
    mAP = 0.0
    for oneclass in range(0,num_classes):
        mAP+=ap[oneclass]
        print('AP for ' + classes.get(oneclass) +" =  " + str(ap[oneclass]*100))

    print('~~~~~~~~~~~~~~~~~~~~~~~')
    mAP /= num_classes
    print('Mean AP@' + str(ov_thresh) +' = ' + str(mAP*100))

    map_all.append(mAP)
    ap_all.append(ap)
                
                

In [ ]:
#######################################################FCIS
all_tp = [[],[],[],[],[]]
all_fp = [[],[],[],[],[]]
overlap_areas = [[],[],[],[],[]]
all_scores = [[],[],[],[],[]]
gt_cls_num = np.zeros(num_classes)

for oneclass in range(0,num_classes):
    print(classes.get(oneclass))
    #find all instances in this class
    for i in range(0,len(fcis_part_all)):
        if i %1000 == 0:
            print(i)
        part = fcis_part_all[i]
        part_label = fcis_part_label_all[i]
        
        gt_part = gt_part_all[i]
        gt_part_label = gt_part_label_all[i]
        
        this_cls =  classes.get(oneclass)
        if this_cls in part_label:
            tp, fp , ov_areas , scores ,gt_thiscls_num= GetAP(part,part_label,gt_part,gt_part_label,oneclass,gt_list[i])
            #print(scores)
            #print(ov_areas)
            all_tp[oneclass].extend(tp)
            all_fp[oneclass].extend(fp)
            overlap_areas[oneclass].extend(ov_areas)
            all_scores[oneclass].extend(scores)
            gt_cls_num[oneclass] = gt_cls_num[oneclass] + gt_thiscls_num

CubeBody
0
1000


In [ ]:
ap = np.zeros(num_classes)
for oneclass in range(0,num_classes):
    
    ####### 按照score重新对tp 和 fp 排序
    orderidx = np.argsort(all_scores[oneclass])
    orderidx = orderidx[::-1]
    tps_backup = all_tp[oneclass][:]
    fpt_backup = all_fp[oneclass][:]
    or_tps = []
    or_fps = []
    for x in orderidx:
        or_tps.append(all_tp[oneclass][x])
        or_fps.append(all_fp[oneclass][x])
        one_tp = or_tps[:]
        one_fp = or_fps[:]
    ##################################

    #one_tp = all_tp[oneclass]
    #one_fp = all_fp[oneclass]
    
    num_pos = gt_cls_num[oneclass]
    one_tp = np.cumsum(one_tp)
    one_fp = np.cumsum(one_fp)
    rec = one_tp / np.maximum(float(num_pos),  np.finfo(np.float64).eps)
    #avoid divide by zero in case the first matches a difficult gt
    prec = one_tp / np.maximum(one_fp+one_tp, np.finfo(np.float64).eps)
    ap[oneclass] = voc_ap(rec, prec, True)
print('FCIS')
print('~~~~~~~~~~~~~~~~~~~~~~~')
mAP = 0.0
for oneclass in range(0,num_classes):
    mAP+=ap[oneclass]
    print('AP for ' + classes.get(oneclass) +" =  " + str(ap[oneclass]*100))
    
print('~~~~~~~~~~~~~~~~~~~~~~~')
mAP /= num_classes
print('Mean AP@' + str(ov_thresh) +' = ' + str(mAP*100))

In [46]:
fcis_dcn_noss_score_backup = all_scores[:]
fcis_dcn_noss_part = fcis_part_all[:]
fcis_dcn_noss_part_label = fcis_part_label_all[:]
fcis_dcn_noss_tp_5 = all_tp[:]
fcis_dcn_noss_fp_5 = all_fp[:]

In [70]:
############################################################################
####save each instance as images

In [28]:
def save_inst(fcis_part_all,fcis_part_label_all,folder):
    os.mkdir(folderpath+folder)
    for i in range(0,len(fcis_part_all)):
        imgname = img_names[i]
        oneimg = fcis_part_all[i]
        oneimg_label = fcis_part_label_all[i]
        for j in range(0,len(oneimg)):
            inst = oneimg[j]
            inst[inst>=1] = 255
            inst_label = oneimg_label[j]
            name = imgname
            filename = folderpath +folder+'/'+ name + '_'+str(j)+'_' + inst_label +'.png'
            print(filename)
            cv2.imwrite(filename, inst)

In [29]:
img_names = image_names
#save_inst(gt_part_all,gt_part_label_all,'gt_split')
#save_inst(fcis_part_all,fcis_part_label_all,'fcis_split')
save_inst(fcis_part_all,fcis_part_label_all,'10_30_FCIS_ob_lr_0005_split')

Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00003_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00003_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00007_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00007_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00010_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00010_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00010_2_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00010_3_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00011_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00011_1_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00011_2_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00011_3_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00013_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bal_00013_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_spl

Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00151_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00158_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00158_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00161_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00161_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00170_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00170_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00171_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00171_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00173_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00173_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00174_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00174_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_00175_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/bla_0

Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00171_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00172_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00172_1_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00172_2_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00174_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00174_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00174_2_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00177_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00177_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00177_2_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00184_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00184_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00185_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00185_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_0

Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00432_2_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00434_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00434_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00436_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00436_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00439_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00439_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00441_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00441_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00446_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00446_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00448_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00448_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/box_00448_2_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_spl

Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00251_2_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00251_3_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00251_4_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00252_0_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00252_1_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00252_2_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00252_3_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00252_4_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00258_0_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00258_1_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00258_2_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00259_0_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00259_1_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cha_00259_2_CylinderF

...

Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00370_1_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00370_2_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00372_0_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00372_1_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00372_2_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00373_0_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00373_1_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00373_2_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00376_0_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00376_1_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00376_2_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00377_0_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00377_1_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/vas_00377_2_CylinderF

Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00459_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00463_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00463_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00464_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00464_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00465_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00465_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00466_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00466_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00467_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00467_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00472_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00472_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_00482_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_00

Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01749_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01749_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01749_2_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01755_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01755_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01758_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01758_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01759_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01759_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01761_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01765_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01768_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01768_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01768_2_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_l

Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01946_2_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01946_3_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01946_4_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01947_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01947_1_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01947_2_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01947_3_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01953_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01953_1_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01953_2_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01958_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01958_1_CylinderFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01967_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_01967_1_CubeBody.png
Exper_

Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02330_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02333_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02334_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02334_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02335_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02335_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02335_2_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02335_3_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02354_0_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02361_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02361_1_CylinderBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02364_0_CubeBody.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02364_1_CubeFace.png
Exper_FCIS_DCN/10_30_FCIS_ob_lr_0005_split/cube_02365_0_CubeBody.png
Exper_FCIS_DCN/10_30_F

In [ ]:
#########################################################

In [ ]:
##############################################################################
####save per image overlap

In [23]:
def PerImageOV(part,label,gt_part,gt_label):
    # part is one image
    ov = np.zeros(len(part))
    gt_detect_list = np.zeros(len(gt_part))
    for i in range(0,len(part)):
        cur_overlap = -1000
        curr_overlap_idx = -1
        for j in range(0,len(gt_part)):
            if gt_label[j] == label[i]:
                ovv = mask_overlap(gt_part[j], part[i])
                if ovv > cur_overlap:
                    cur_overlap = ovv
                    curr_overlap_idx = j
        if cur_overlap == -1000:
            ov[i] = 0
        else:
            if gt_detect_list[curr_overlap_idx]==0:
                ov[i] = cur_overlap
                gt_detect_list[curr_overlap_idx]==1
            else:
                ov[i] = 0
    return ov

In [24]:
counter = 0
fcis_mean_ov = 0
for i in range(0,len(fcis_part_all)):
    #print(i)
    ov = PerImageOV(fcis_part_all[i],fcis_part_label_all[i],gt_part_all[i],gt_part_label_all[i])
    print(ov)
    for x in ov:
        counter = counter + 1
        fcis_mean_ov = x + fcis_mean_ov    
    #filename = fcis_list[i]
    #savefile(folderpath+'fcis/'+filename[:filename.index('.')]+'.txt',ov)

[ 0.65633475  0.        ]
[ 0.2268159]
[ 0.         0.         0.8578362]
[ 0.          0.83039648]
[ 0.854035    0.93994082  0.38944717]
[ 0.  0.  0.  0.  0.  0.]
[]
[ 0.]
[ 0.  0.]
[ 0.88254629]
[ 0.  0.  0.]
[ 0.]
[ 0.92394334  0.94042572]
[ 0.847061    0.42210667  0.49913993]
[ 0.93086817  0.9244508 ]
[ 0.88640174  0.9313855 ]
[ 0.78910798  0.89674101]
[ 0.94015342  0.89470025]
[ 0.76597422  0.71728992]
[ 0.82746927  0.88383205]
[ 0.]
[ 0.8146519   0.77155701]
[ 0.9117041   0.88244986]
[ 0.  0.  0.  0.]
[ 0.85152517  0.93329168]
[ 0.89063555  0.91015096  0.64011856  0.62854331]
[ 0.         0.         0.         0.6589698]
[ 0.  0.  0.]
[ 0.33955438  0.        ]
[ 0.92060165  0.94998108]
[ 0.87027353]
[ 0.61251139]
[ 0.94455703  0.93239158]
[ 0.76355113  0.        ]
[ 0.  0.]
[ 0.9565762   0.86868742]
[ 0.59809524  0.          0.79316338]
[ 0.  0.  0.]
[ 0.          0.          0.72402419]
[ 0.01028504  0.5402867   0.34503511]
[ 0.         0.1224391]
[ 0.  0.]
[ 0.          0.     

[ 0.10155908  0.57458162  0.66603641]
[ 0.]
[ 0.92564821  0.91039042]
[ 0.  0.]
[ 0.         0.         0.7237112]
[ 0.67974649]
[ 0.02078378]
[ 0.87783661  0.94946145]
[ 0.69981527  0.89473929]
[ 0.94463351  0.87895546]
[ 0.82978598  0.94060735]
[ 0.19965572]
[ 0.]
[ 0.         0.         0.8222406]
[ 0.75887691  0.90887393]
[ 0.          0.          0.          0.87526055]
[ 0.          0.79269016]
[ 0.          0.          0.84184823  0.          0.        ]
[ 0.87649078  0.82990084]
[ 0.94104301  0.00698617]
[ 0.89759937  0.68680472]
[ 0.  0.  0.]
[ 0.92302746  0.93367706]
[ 0.86221837  0.91172125]
[ 0.          0.          0.91698924]
[ 0.          0.          0.73590316  0.85227816]
[ 0.          0.          0.          0.69634862]
[ 0.  0.  0.]
[ 0.60495522  0.74917492  0.88497557  0.77192685  0.86723677]
[ 0.95093624  0.91757878]
[ 0.  0.  0.  0.]
[ 0.91440956  0.84293955]
[ 0.          0.89540059]
[ 0.8727133   0.72661576  0.05651309]
[ 0.          0.87123288]
[ 0.73638968  0.

[ 0.51156495  0.45092025  0.74352367]
[ 0.94719017  0.9388322 ]
[ 0.94292913  0.93241806]
[ 0.95178566  0.92226633]
[  8.74655225e-01   2.83173812e-05   8.89420035e-01]
[ 0.91938216  0.8758209 ]
[ 0.95246754  0.94283554]
[ 0.         0.         0.         0.4375975]
[ 0.  0.]
[ 0.  0.]
[ 0.94855352  0.95153506]
[]
[ 0.          0.          0.64415387  0.4673256 ]
[ 0.76178997  0.95211688]
[ 0.9506228   0.95422128]
[ 0.]
[]
[ 0.  0.]
[ 0.88014383  0.9265506 ]
[ 0.92721843  0.56935687]
[ 0.]
[ 0.]
[]
[ 0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.]
[ 0.  0.  0.]
[ 0.19944598  0.709458    0.75311651]
[ 0.90623726  0.96015969]
[ 0.]
[ 0.  0.]
[ 0.8462847   0.85456148  0.87501147  0.77202177]
[ 0.]
[ 0.         0.1135165  0.700625 ]
[ 0.714791    0.94154403]
[]
[ 0.56601307  0.921262  ]
[ 0.93829504  0.94664752]
[ 0.]
[ 0.88807683  0.88503399]
[ 0.89662677  0.90153779]
[ 0.94017592  0.91855726]
[ 0.  0.]
[ 0.          0.          0.80878534  0.70619776]
[ 0.50152905  0.96989308]
[ 0.  0.]
[ 0.      

[ 0.7474414  0.       ]
[ 0.  0.  0.]
[ 0.77809147  0.8394209 ]
[ 0.76689446  0.95078969]
[ 0.91211443  0.90409743]
[ 0.          0.          0.85770841]
[ 0.80995666  0.86959031]
[ 0.  0.]
[ 0.91961399  0.8989138 ]
[ 0.          0.          0.          0.92876929]
[ 0.          0.          0.90813911  0.9025695 ]
[ 0.  0.]
[ 0.43338584]
[ 0.91839992  0.89751027]
[ 0.92424242  0.94157759]
[ 0.68635494  0.83307573]
[ 0.]
[ 0.89973115  0.88770842]
[ 0.          0.          0.          0.          0.83943089  0.84099617]
[ 0.  0.  0.]
[ 0.09250109  0.07908376  0.        ]
[ 0.  0.  0.  0.]
[ 0.57659128  0.78664424]
[ 0.  0.]
[ 0.          0.          0.          0.76722133]
[ 0.95473428  0.9225923 ]
[ 0.86676893]
[ 0.          0.          0.79326701]
[ 0.93009915  0.8990954 ]
[ 0.76206724  0.84192298]
[ 0.77891216  0.71519443]
[ 0.]
[ 0.88989057  0.83071931]
[ 0.]
[ 0.8928278   0.02419001]
[ 0.90811674  0.9238002   0.73355956]
[ 0.94424278  0.95209903]
[ 0.]
[]
[ 0.91991716]
[ 0.91311086 

[ 0.         0.7245509  0.       ]
[]
[ 0.]
[ 0.          0.91034913]
[ 0.57729576  0.70288079]
[ 0.92450204  0.94273054]
[ 0.  0.]
[ 0.01378868  0.        ]
[ 0.  0.]
[ 0.87124464  0.91332888]
[ 0.71668956  0.          0.78964942]
[ 0.  0.]
[ 0.          0.          0.73170732]
[ 0.  0.]
[ 0.  0.  0.]
[]
[ 0.          0.          0.55234657]
[ 0.  0.  0.  0.]
[ 0.  0.  0.]
[ 0.90592972  0.63560034]
[ 0.91515391  0.92275208]
[ 0.  0.]
[ 0.73680782  0.92880863]
[ 0.54466403  0.90832063]
[ 0.          0.          0.82321188  0.82360529  0.          0.        ]
[ 0.93271462  0.95200259]
[ 0.19850352]
[ 0.]
[ 0.          0.83003184]
[ 0.00639397  0.09404117]
[]
[ 0.91136803  0.72253993]
[ 0.  0.  0.  0.]
[ 0.0068262   0.16074864]
[ 0.  0.  0.  0.  0.]
[ 0.84481715  0.94511246]
[ 0.          0.          0.82979452]
[ 0.          0.          0.4461057   0.38398177]
[ 0.91200342  0.90438717]
[ 0.92275997  0.93283344]
[]
[ 0.9265796   0.91712573]
[ 0.]
[ 0.          0.          0.80132309]
[ 0

...

[ 0.92892114  0.94526564]
[ 0.  0.]
[ 0.          0.          0.          0.90333158]
[ 0.57931625  0.80592654  0.97060406]
[ 0.94293118  0.95381685]
[ 0.]
[ 0.         0.         0.         0.         0.         0.7486965]
[ 0.  0.  0.]
[ 0.          0.          0.66610169  0.65840708]
[ 0.90538807  0.92077857]
[ 0.          0.          0.56570672]
[ 0.  0.]
[ 0.82891285  0.93702165]
[ 0.  0.  0.]
[ 0.          0.8081903   0.77986023  0.          0.        ]
[]
[ 0.  0.]
[ 0.]
[ 0.  0.  0.]
[ 0.  0.  0.]
[ 0.76320378  0.63926262]
[ 0.56868795  0.91996927]
[ 0.  0.  0.]
[ 0.          0.92252102]
[ 0.  0.]
[ 0.          0.          0.62249428]
[ 0.90663043  0.87982011]
[ 0.66263007  0.78843534  0.8770201 ]
[ 0.          0.          0.69247432]
[ 0.          0.          0.          0.80246092]
[ 0.45883642]
[ 0.88525495  0.95039041]
[ 0.          0.          0.72433853]
[ 0.73413109  0.56024707]
[]
[ 0.93977901  0.95511487]
[ 0.90575472  0.7538137 ]
[ 0.  0.]
[]
[ 0.01156677  0.51925667]

[ 0.  0.  0.  0.  0.]
[ 0.001853    0.59153036]
[ 0.  0.  0.  0.]
[ 0.          0.          0.84961619]
[ 0.]
[ 0.  0.  0.]
[ 0.          0.76087336]
[ 0.78823529  0.88036847]
[ 0.90505695  0.88146796]
[ 0.83289538  0.9063426   0.65059113  0.78599171]
[ 0.          0.40802213  0.          0.48316062  0.83335949]
[ 0.90273849  0.88769656]
[ 0.88194052  0.        ]
[ 0.72163644  0.          0.        ]
[ 0.95771151  0.94213218]
[ 0.93356101  0.9375    ]
[ 0.8622975   0.85714286  0.83320979  0.90667945  0.86587302  0.88881748
  0.82972136  0.85577841  0.90193506  0.8896481 ]
[ 0.61584924]
[ 0.  0.]
[ 0.74279413  0.45008723]
[ 0.23742561  0.88627668]
[ 0.  0.  0.  0.]
[ 0.04948467]
[ 0.  0.]
[ 0.          0.          0.91693009]
[ 0.  0.]
[ 0.          0.62867012  0.59791123  0.          0.8608522 ]
[ 0.          0.          0.7538674   0.          0.75672737  0.
  0.55834464  0.77358491]
[ 0.          0.          0.78108652]
[ 0.91871567  0.70625541]
[ 0.  0.]
[ 0.  0.]
[ 0.51392703]
[ 0.

[ 0.94448617  0.94906612]
[ 0.87793255  0.90730306]
[ 0.]
[ 0.          0.          0.79432283]
[ 0.92432044  0.92424455]
[ 0.85330619]
[ 0.          0.          0.91778793]
[ 0.         0.6955267]
[ 0.95322456  0.90363563]
[ 0.85384869  0.91231793]
[ 0.80339335  0.52654806]
[ 0.91440589  0.94617429]
[ 0.85186979  0.90213261]
[ 0.83157895  0.92537961  0.81152153  0.7719797 ]
[ 0.          0.          0.79468098  0.60830527]
[ 0.2127712]
[ 0.36569482  0.50312278]
[ 0.          0.          0.76720383]
[ 0.01240038]
[ 0.  0.  0.]
[ 0.]
[ 0.96276052  0.96423643]
[ 0.          0.78810409]
[ 0.79832854  0.          0.81087342  0.92901054]
[ 0.  0.  0.  0.  0.  0.]
[ 0.85884597  0.93071805]
[ 0.  0.]
[ 0.82351852  0.54209919  0.87793081  0.71799333  0.92797235]
[ 0.          0.          0.69395263  0.71546547]
[ 0.92929964  0.94437931]
[ 0.90598925  0.9167005 ]
[ 0.          0.          0.          0.65132188]
[ 0.82921404  0.          0.        ]
[ 0.95041737  0.94935971]
[ 0.          0.   

[ 0.78132716  0.8264922   0.69468522  0.83168883  0.8421362   0.74149719]
[ 0.  0.  0.]
[ 0.715       0.43196106]
[ 0.  0.  0.]
[ 0.87429719  0.88167748]
[ 0.7856305   0.80138356]
[ 0.89557425]
[ 0.  0.  0.]
[ 0.9042525   0.79996946]
[ 0.64596162]
[ 0.          0.78891616  0.96494089]
[ 0.]
[ 0.          0.          0.49576562]
[ 0.          0.          0.86073574]
[]
[ 0.          0.57592154  0.64067914  0.          0.93242187]
[ 0.          0.70921986]
[ 0.          0.          0.          0.84041184]
[ 0.          0.          0.          0.91204589]
[ 0.          0.          0.91569628]
[ 0.          0.          0.38320995]
[ 0.  0.]
[ 0.          0.58657613]
[ 0.94001751  0.91963048]
[ 0.89592998  0.82428168]
[ 0.]
[ 0.          0.          0.83615085]
[ 0.  0.]
[ 0.92327044  0.81393868]
[ 0.  0.  0.  0.]
[ 0.  0.]
[ 0.          0.80327528  0.        ]
[ 0.22895395]
[ 0.          0.          0.71697866]
[ 0.          0.          0.          0.80041524  0.84874059]
[ 0.00230909  0. 

[ 0.95116755  0.91624638]
[ 0.93237864  0.92017536]
[ 0.94816812  0.93580229]
[ 0.          0.          0.58735699  0.76277017]
[ 0.76769626]
[]
[ 0.09248848]
[ 0.  0.  0.  0.  0.]
[ 0.          0.          0.53007519  0.82315064]
[ 0.90161991  0.95957011]
[ 0.78699734  0.90353644]
[ 0.]
[ 0.86582084  0.88249922]
[ 0.  0.  0.]
[ 0.          0.          0.70423163  0.68654482]
[ 0.00094829  0.86540711]
[ 0.93917949  0.90514517]
[ 0.93172578  0.93735172]
[ 0.49044869  0.        ]
[ 0.92724523  0.88268233]
[ 0.          0.          0.84434762]
[ 0.  0.]
[ 0.6471364]
[]
[ 0.4572532   0.42671255  0.81421791]
[ 0.91667617  0.95320901]
[ 0.  0.]
[ 0.80182232  0.96418323]
[ 0.]
[ 0.  0.]
[ 0.57180414  0.92933492]
[ 0.          0.          0.42039106]
[ 0.  0.]
[ 0.85567672  0.82210243]
[ 0.          0.          0.88313062]
[ 0.          0.          0.71010542]
[ 0.95969905  0.96537458]
[ 0.37102459  0.        ]
[ 0.90906314  0.92807902]
[ 0.91330588]
[ 0.          0.          0.73295198]
[ 0.7

In [25]:
print(fcis_mean_ov)
print(counter)
print(fcis_mean_ov/(2*counter))

3514.61452873
8355
0.210330013688


In [32]:
counter = 0
geonet_mean_ov = 0
imgovlist = []
for i in range(0,len(geonet_part_all)):
    #print(i)
    ov = PerImageOV(geonet_part_all[i],geonet_part_label_all[i],gt_part_all[i],gt_part_label_all[i])
    if 'CylinderBody' in geonet_part_label_all[i]:
        imgovlist.append(ov.mean())
        print(ov)
    else:
        imgovlist.append(0)
    for x in ov:
        counter = counter + 1
        geonet_mean_ov = x + geonet_mean_ov
        
    #filename = geonet_list[i]
    #savefile(folderpath+'geonet/'+filename[:filename.index('.')]+'.txt',ov)

[ 0.50966481  0.79569456  0.83667883  0.84591008]
[ 0.69959862  0.93715898  0.80275892]
[ 0.96122158  0.91028876]
[ 0.57677903  0.33146067  0.8300123   0.89010283  0.87571893  0.52678571
  0.74451411]
[ 0.60400407  0.90558369  0.71456486]
[ 0.03359986  0.          0.05113863  0.13390207]
[ 0.62285273  0.7975776   0.        ]
[ 0.80239521  0.93168626  0.81852753  0.51553484]
[ 0.92695288  0.74889734  0.82586922  0.8783952 ]
[ 0.63139932  0.37595908  0.79815016  0.42119565  0.86476523  0.
  0.04471545]
[ 0.0357922   0.8302583   0.88393852  0.57092496  0.00342857]
[ 0.93849613  0.60744451  0.66900621  0.72157153  0.        ]
[ 0.16779067  0.          0.89081747  0.87615601]
[ 0.67360645  0.81848875  0.01775457  0.97279811  0.86763479  0.74743076]
[ 0.57436276  0.69156515]
[ 0.63009404  0.76841778  0.70883755  0.53562082  0.88762737  0.87751371]
[ 0.          0.87550882  0.          0.72321429]
[ 0.79727736  0.81809485  0.68692372  0.74046961]
[ 0.9515636   0.84839238  0.11667439]
[ 0.    

[ 0.87769357  0.89169288  0.81668336  0.73824724]
[ 0.80603843  0.81946732  0.46006262]
[ 0.57971014  0.88660742  0.68068021  0.7635028   0.7585335   0.32669592
  0.84326923  0.62591067  0.24290221]
[ 0.71410929  0.90713811  0.86521377]
[ 0.75248381  0.85518835  0.50138497]
[ 0.00642769  0.8716233   0.78358789  0.85058035  0.54291819  0.61284567
  0.46615769  0.65492745  0.63764479]
[ 0.72988104  0.84107286  0.49929677  0.89018975]
[ 0.68156109  0.          0.74746699  0.79755829  0.63755013  0.7834385
  0.6931661 ]
[ 0.73577982  0.89563541  0.        ]
[ 0.65051903  0.94220183  0.93798988  0.81488473  0.73797282  0.73411838
  0.44825376]
[ 0.88180113  0.66666667  0.76515152  0.60243408  0.87180723]
[ 0.31029847  0.          0.72682927]
[ 0.89122807  0.9718946   0.83681399  0.82166247  0.78606692]
[ 0.69460803  0.83886389  0.80959582  0.83296214]
[ 0.70653256  0.65480696  0.10794894  0.80545571  0.80456656]
[ 0.89121339  0.88748287  0.2620539 ]
[ 0.51068534  0.          0.90745624  0.9

[ 0.78969774  0.90360199  0.          0.82150376]
[ 0.75547425  0.58508957  0.01583235  0.69690011  0.71990307]
[ 0.63994222  0.91251993  0.83451129]
[ 0.62175765  0.04597701  0.39918534  0.90033317  0.89878163]
[ 0.80126423  0.86005421  0.60651587]
[ 0.5152439   0.89439853  0.75055782]
[ 0.60935857  0.70676692  0.93167631  0.91634063  0.82238866  0.9268619 ]
[ 0.59079284  0.          0.77684324  0.84450294  0.6978332   0.77534006]
[ 0.82135186  0.78490893  0.86775526]
[ 0.5519495   0.          0.61873007  0.61392072  0.94316715  0.6901726 ]
[ 0.03085794  0.51683081  0.83638346  0.75297144]
[ 0.81600832  0.91280366  0.89191489  0.87610472  0.9313622   0.70192726]
[ 0.90465296  0.69954835]
[ 0.          0.47738693  0.78806584  0.        ]
[ 0.          0.          0.7381717   0.89661882  0.92699758]
[ 0.72703973  0.51724727  0.57017544]
[ 0.79904824  0.90593552  0.73192436  0.75804575  0.81794498]
[ 0.93310507  0.9380989   0.          0.60180523]
[ 0.87497249  0.          0.70390782]
[ 

[ 0.83777365  0.81644709  0.69535881]
[ 0.]
[ 0.55264578  0.          0.86056854]
[ 0.83344972  0.93878772  0.512303    0.7116816   0.77859949]
[ 0.92921493  0.92777971  0.81030702  0.91074596  0.7107608   0.72989172]
[ 0.93979163  0.80860801  0.        ]
[ 0.58386838  0.86470897  0.75058324  0.30117108]
[ 0.79159105  0.79966997  0.8959276   0.        ]
[ 0.88256387  0.96540122  0.86237374]
[ 0.91623667  0.60920831  0.90119698  0.86705989]
[ 0.88625904  0.7664042   0.6112577   0.90973843  0.8600242 ]
[ 0.8078853   0.8369338   0.60151007  0.83212268]
[ 0.88637261  0.77944496  0.7793563   0.57516834]
[ 0.94520366  0.74963181  0.82554862  0.88781897]
[ 0.84740154  0.70562454]
[ 0.53899309  0.84147575  0.          0.47907099]
[ 0.57698962  0.88373847  0.79738872  0.57755923  0.82088096  0.48564103
  0.12601532  0.41225242]
[ 0.86878453  0.88753094  0.85401174  0.89878105  0.7825905   0.82716049
  0.77087287]
[ 0.74328842  0.        ]
[ 0.94358732  0.90878378  0.79458863  0.71413102  0.8399

[ 0.82827039  0.81528055  0.73874126  0.67626722  0.74137003  0.65683539]
[ 0.          0.82245253  0.          0.          0.80145981]
[ 0.71580817  0.89259824  0.48797737  0.65174825]
[ 0.52601432  0.61659389  0.57896472  0.76961353  0.56934026]
[ 0.00323104  0.62698413  0.88263572  0.58204334  0.          0.        ]
[ 0.92949757  0.85438972]
[ 0.92907496  0.85196248  0.71106434]
[ 0.95045391  0.8784141   0.81783587  0.60475884]
[ 0.00438711  0.76137113]
[ 0.73113381  0.89128976  0.81132563]
[ 0.89994272  0.79456849]
[ 0.32271681  0.79649583  0.93587084  0.7311344   0.          0.          0.        ]
[ 0.62337662  0.90651801  0.89510007  0.78777819  0.52545249]
[ 0.90467472  0.96932409  0.84180871  0.91198623]
[ 0.38595401  0.92889634  0.36793848]
[ 0.68947018  0.88288509]
[ 0.          0.40328791  0.90001462  0.89873282  0.93548247  0.91466062
  0.66730465]
[ 0.94658449  0.63729746  0.84630292  0.28326679]
[ 0.6751269   0.59544749  0.64692321  0.90975884  0.89762115]
[ 0.69728601 

...

[ 0.80868842  0.81016206  0.83323039  0.68100713  0.59141247]
[ 0.96888498  0.78752086  0.8707269   0.87983418]
[ 0.95211609  0.09101494  0.90185827  0.82625164  0.          0.81173491
  0.83903302  0.48716343]
[ 0.73827328  0.77278426  0.88610285  0.94288496  0.77064362  0.82918319]
[ 0.          0.93148447  0.47511291  0.        ]
[ 0.75458538  0.8171123   0.31519346  0.60827251]
[ 0.625       0.95161807  0.79904306]
[ 0.62920489  0.88308113  0.84717198  0.76159779  0.39358188  0.78721238]
[ 0.88073394  0.80400798  0.53133469  0.86382979]
[ 0.55528667  0.87576119  0.79384944  0.7159581 ]
[ 0.91922631  0.86121324  0.73732719  0.67488912  0.        ]
[ 0.34867197  0.70342707  0.73636225  0.85437816  0.73368921  0.88548057]
[ 0.          0.          0.          0.85358005]
[ 0.59785638  0.82256077  0.9151463   0.31785817]
[ 0.83822339  0.92779414  0.81575173]
[ 0.62541413  0.92898226  0.34101767]
[ 0.67825575  0.95418363  0.8360302 ]
[ 0.          0.83793911  0.76611781  0.94329518  0.7

[ 0.61543799  0.67480243  0.          0.        ]
[ 0.92817995  0.89002157  0.74842371]
[ 0.58088235  0.95653516  0.75509017]
[ 0.86764935  0.52526122  0.89752504]
[ 0.          0.91193331  0.58735689  0.9463586 ]
[ 0.71980609  0.87353151  0.02809334  0.00980249  0.39947061]
[ 0.85386177  0.63236299  0.82290554  0.83915321  0.334305    0.51782499]
[ 0.87021909]
[ 0.61967695  0.70176335  0.6446381   0.87367297  0.85480349]
[ 0.75342466  0.87109712  0.        ]
[ 0.74744514  0.        ]
[ 0.80885569  0.93013348  0.63596059]
[ 0.584       0.92670342  0.          0.90210843]
[ 0.          0.83173202  0.          0.81787407  0.80608365  0.89731463
  0.89884148  0.80879121]
[ 0.93112098  0.78826165  0.93940511  0.83354997  0.83542191  0.89054373]
[ 0.9098863   0.          0.82887267]
[ 0.94314104  0.69386348  0.46372068]
[ 0.67411062  0.87646207  0.81137993]
[ 0.25316456  0.78157163  0.89105582  0.80911396  0.86671418  0.        ]
[ 0.7700813   0.90791234  0.86694476]
[ 0.5503265   0.6268257

[ 0.50959898  0.3724981   0.86019494  0.73174891  0.          0.11538462]
[ 0.84302789  0.89462844  0.          0.74536432]
[ 0.91118548  0.62570073  0.85410085]
[ 0.          0.84310279  0.85698866  0.60436351]
[ 0.84799107  0.86773184  0.89755223  0.90951572]
[ 0.48133878  0.          0.        ]
[ 0.75544794  0.88708838  0.        ]
[ 0.          0.86748481  0.96774308  0.86104061]
[ 0.14103261  0.9013274   0.93893484  0.88403382  0.8609358   0.73706708]
[ 0.00529714  0.          0.74618488  0.          0.          0.        ]
[ 0.86736313  0.8898857   0.77086304]
[ 0.84747374  0.82638196  0.87285201  0.51340338  0.          0.        ]
[ 0.81290909  0.85932246  0.07039297  0.77593864  0.72877774  0.88805031]
[ 0.89133517  0.90465628  0.          0.        ]
[ 0.48844521  0.82395269  0.63612663  0.33627451]
[ 0.82659356  0.83200174  0.89928315  0.77346437]
[ 0.76583439  0.917481    0.88795968]
[ 0.54235493  0.62721295  0.84265166  0.77102409  0.76931761  0.56722689]
[ 0.66666667  0.

[ 0.90551469  0.90074357]
[ 0.53834116  0.4707271   0.77536646  0.3833581   0.89985302  0.81458495
  0.92924177  0.7696      0.82350629  0.78792822]
[ 0.87002505  0.61872288  0.66731518  0.88135717  0.78904429]
[ 0.          0.85647376]
[ 0.48193305  0.8028991   0.92250712  0.61811089]
[ 0.88206398  0.86235326  0.80927712  0.76782822  0.60428987]
[ 0.          0.87382893  0.          0.          0.        ]
[ 0.9194526   0.52397923  0.2961022   0.00352003  0.24141106]
[ 0.          0.91005282  0.77433462  0.84152255]
[ 0.74106324  0.81394479]
[ 0.84113475  0.85158767  0.68345324]
[ 0.85690516  0.89058524  0.47520661  0.38762887  0.89383562]
[ 0.7521616   0.57396136]
[ 0.89590871  0.87760703  0.8593435   0.86872524  0.67667292  0.67283163
  0.64041514  0.64444444  0.6899652   0.68612079]
[ 0.70799104  0.7004519   0.89287808  0.85196791  0.68119748]
[ 0.81950121  0.9294342 ]
[ 0.95888771  0.8836478 ]
[ 0.75199169  0.91602791  0.79255542  0.71060456]
[ 0.43538136  0.72329142  0.91308523]


[ 0.94480551  0.6901888   0.85620374  0.53770919  0.90544321  0.78346457]
[ 0.8819024   0.94838599  0.90734234  0.74600773  0.89109182  0.77207763
  0.83143996  0.7574281 ]
[ 0.77490329  0.91717879  0.03335974]
[ 0.62913415  0.65827762  0.45404412  0.69687109  0.77976025]
[ 0.72259949  0.          0.69445747]
[ 0.85273492  0.86658688  0.61022928  0.          0.        ]
[ 0.09787234  0.86361057  0.87909447  0.87624193  0.93710366  0.87141467
  0.54484189  0.21173763]
[ 0.84301413  0.77737973]
[ 0.3737929  0.       ]
[ 0.87283844  0.          0.54428874]
[ 0.86470129  0.        ]
[ 0.85985968  0.85831205  0.97323119  0.9531635   0.          0.
  0.80963353  0.88506429]
[ 0.          0.95958109  0.39615129  0.73197674]
[ 0.          0.95183135  0.49219949  0.86599692  0.7541424   0.64795642]
[ 0.71254281  0.94082678  0.81366751  0.77379553]
[ 0.93317193  0.89524261]
[ 0.82330591  0.91387706  0.61803813]
[ 0.91349028  0.88964483  0.94748968  0.76043768  0.91367193  0.93096989]
[ 0.4416788

In [33]:
print(geonet_mean_ov)
print(counter)
print(geonet_mean_ov/counter)

7695.32689213
11455
0.67178759425


In [34]:
####################################################################

In [35]:
print(len(imgovlist))
print(len(geonet_part_all))

3474
3474


In [36]:
imglist = fcis_list[:]
orderidx = np.argsort(imgovlist)
orderidx = orderidx[::-1]
or_imglist = []
for x in orderidx:
        print(imgovlist[x])
        or_imglist.append(imglist[x])
    ################################## 

0.95609444536
0.95303878409
0.949853171517
0.948022436459
0.947427151226
0.946105273665
0.945370638421
0.94524424757
0.943372707307
0.940775411937
0.939266742332
0.935984416273
0.935755171421
0.935665457855
0.935027694733
0.934644297786
0.932776357664
0.932772270607
0.93260185062
0.930314002493
0.929854520022
0.92791162419
0.927771582494
0.926039041816
0.925983268076
0.925915090032
0.924913077689
0.924046505298
0.923429903646
0.923002926155
0.921267752484
0.92111355833
0.920926424143
0.920712578598
0.920534261708
0.919235470374
0.914626178789
0.914243653169
0.914207274531
0.912971363682
0.912689849034
0.912645950046
0.912254084684
0.911547408768
0.911302814825
0.910992679592
0.910290618218
0.910097527684
0.909193365243
0.908258222779
0.907853864375
0.906948435556
0.906518425887
0.90587072897
0.90503122554
0.904837004828
0.90434985945
0.904137026038
0.903783567172
0.903758445861
0.903446278005
0.90312912899
0.902432390216
0.902308422173
0.902215387483
0.901577676065
0.901548671594
0.901

0.733643748116
0.733599105515
0.733294720465
0.732669154039
0.731947551382
0.731624021875
0.731030433
0.731029763995
0.730755414181
0.730130100559
0.729870464425
0.729559831478
0.729286030196
0.729221932708
0.728855986244
0.728846918968
0.727971719962
0.727371223982
0.727112990096
0.726801350143
0.726727882344
0.726546203283
0.726523811604
0.726082837756
0.72489213563
0.724400517037
0.7241752134
0.722876255168
0.722842482288
0.722477768805
0.722444850885
0.722378338818
0.722299148173
0.721994801911
0.721926784874
0.720606451729
0.720066751412
0.719357451971
0.719296514986
0.719250706002
0.718924417447
0.718455379875
0.717804555185
0.716975007672
0.71663159441
0.716451514916
0.715946810555
0.715900117698
0.715517368263
0.71462253715
0.713826955403
0.713641152437
0.713626322988
0.713515456151
0.713356780389
0.713002535244
0.712930777646
0.712757296265
0.712554856267
0.712387874649
0.712056244949
0.711837337142
0.711284941907
0.710638708244
0.710545895687
0.710539260217
0.710334870704
0.7

0.485866083994
0.485361736027
0.483691036693
0.482997078131
0.482287847343
0.481851884399
0.481414615177
0.48123177292
0.480647954874
0.480535204222
0.479733272553
0.479535586975
0.479435746297
0.479408657942
0.479200633687
0.478385151772
0.477787050566
0.47650223866
0.476316141314
0.47621371713
0.475732658448
0.475211605931
0.474893897206
0.474771042812
0.4746182295
0.473476775877
0.473334615021
0.472907921928
0.472874289061
0.472598088036
0.472375724621
0.47235231852
0.472304308829
0.472186937431
0.471470771007
0.471071442097
0.470401685094
0.470250255363
0.469707469446
0.469351644366
0.469158060352
0.468887972801
0.46887643243
0.467221789064
0.466786431932
0.466281315903
0.465910215339
0.464884957911
0.46486851217
0.464436599042
0.462127906397
0.461702190463
0.461638310065
0.461555474746
0.461483862651
0.460350754566
0.460349621201
0.460312746899
0.459947809302
0.459592127628
0.459316673934
0.45867216558
0.457936974995
0.457696291645
0.457363881725
0.456843865447
0.456744316534
0.45

In [40]:
import shutil

folderpath = 'Exper_FCIS_DCN/'

for img in or_imglist[0:100]:
    ourmask = img[:img.index('.')] +'_mask.png'
    fcismask = img
    oriimg =  img[:img.index('.')] +'.jpg'
    gtimg =  img
    
    if os.path.exists(folderpath+'10_30_FCIS_DCN/'  + ourmask):  
        shutil.copyfile(folderpath+'10_30_FCIS_DCN/'  + ourmask,folderpath+'compbnf/ours/'+ourmask)
        shutil.copyfile(folderpath+'FCIS_baseline/' +  fcismask,folderpath+'compbnf/baseline/'+fcismask)
        shutil.copyfile(folderpath+'img/' + oriimg,folderpath+'compbnf/img/'+oriimg)
        shutil.copyfile(folderpath+'gt/' + gtimg,folderpath+'compbnf/gt/'+gtimg)
    